In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'NanumGothic'

matplotlib.rc("axes", unicode_minus = False)

In [3]:
df1 = pd.read_csv("01 Oxidation.csv")
df2 = pd.read_csv("02 Photo_softbake.csv")
df3 = pd.read_csv("03 Photo_lithograpy.csv")
df4 = pd.read_csv("04 Etching.csv")
df5 = pd.read_csv("05 Ion_Implantation.csv")
df6 = pd.read_csv("06 Inspect.csv")

In [4]:
df = pd.merge(df1, df2, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df3, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df4, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df5, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df6, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])

In [5]:
df.dropna(subset="Thin F2", inplace=True) # 다른 변수들과의 연관성을 찾지못함, 한 행에 여러 열들 결측값 가짐
# pd.set_option('display.max_row', 200)
# pd.set_option('display.max_columns', 200)

In [6]:
df['Ox_Chamber'] = df['Ox_Chamber'].astype('str')
df['photo_soft_Chamber'] = df['photo_soft_Chamber'].astype('str')
df['lithography_Chamber'] = df['lithography_Chamber'].astype('str')
df['Etching_Chamber'] = df['Etching_Chamber'].astype('str')
df['Chamber_Num'] = df['Chamber_Num'].astype('str')
df['path'] = df['Ox_Chamber']+df['photo_soft_Chamber']+df['lithography_Chamber']+df['Etching_Chamber']+df['Chamber_Num']

In [7]:
df.drop(df[df['Oxid_time']<0].index, inplace=True) # 산화시간이 음수
df.drop(df[df['Target']==0].index, inplace=True) # target 값이 0
df.drop(columns = ["Wafer_map","Error_message"],inplace=True) # 웨이퍼맵, 에러메시지

In [8]:
# 시간 데이터 datetime 유형으로 변환
df['Datetime'] = pd.to_datetime(df['Datetime'], format='%d-%m-%Y')

# 195이상 = 불량(1), 195미만 = 양품(0)
df.loc[df['Target'] >= 195, '불량_195이상'] =1
df.loc[df['Target'] < 195, '불량_195이상'] =0

In [9]:
df.loc[df['Thin F4']<0, 'Thin F4']=df['Thin F4'].median()
df.loc[df['Flux90s']<0, 'Flux90s']=df['Flux90s'].median()
df.loc[df['Flux160s']<0, 'Flux160s']=df['Flux160s'].median()
df.loc[df['Flux160s']<5, 'Flux160s']=df['Flux160s'].median()

In [10]:
df['Thin F1'].fillna(df['Thin F1'].median(), inplace=True)
df['Thin F3'].fillna(df['Thin F3'].median(), inplace=True)
df['Flux60s'].fillna(df['Flux90s'].median(), inplace=True)
df['Flux90s'].fillna(df['Flux90s'].median(), inplace=True)
df['Flux480s'].fillna(df['Flux90s'].median(), inplace=True)
df['Flux840s'].fillna(df['Flux90s'].median(), inplace=True)

### 불필요한 열 제거

In [11]:
# 불필요한 열 제거 
# df.drop(columns=["No_Die", "Lot_Num", "Wafer_Num"], inplace=True)
df.drop(columns=['Vapor','process','Wavelength'])
df.loc[df['Flux840s'] == 8.137500e+16, 'Flux840s'] = df['Flux840s'].mode()[0]
df.loc[df['Flux480s'] == 8.137500e+16, 'Flux480s'] = df['Flux480s'].mode()[0]

In [12]:
df_new = df[['Temp_OXid','ppm','Pressure','type','Oxid_time',
        'N2_HMDS', 'pressure_HMDS', 'temp_HMDS', 'temp_HMDS_bake',
       'time_HMDS_bake', 'spin1', 'spin2', 'spin3', 'photoresist_bake',
       'temp_softbake', 'time_softbake', 'UV_type', 'Energy_Exposure',       
        'Temp_Etching', 'Source_Power',
       'input_Energy', 'Temp_implantation', 'Furance_Temp', 'RTA_Temp',
       '불량_195이상', 'path']]

In [13]:
# 데이터 구성:Series, DataFrame
import pandas as pd
# 데이터 시각화
import matplotlib.pyplot as plt
import matplotlib

# 데이터 분할:train, test
from sklearn.model_selection import train_test_split
# 분류 Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
# 최적 모델, 파라미터 탐색
from sklearn.model_selection import GridSearchCV

# 분류모델 평가 함수
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report

In [15]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
# LabelEncoder 객체 생성
label_encoder = LabelEncoder()

df_raw_x = df_new.drop("불량_195이상", axis = 1)
df_raw_y = df_new["불량_195이상"] 

# MinMaxScaler 객체 생성
numeric_columns = df_raw_x.select_dtypes(include=['number']).columns
scaler = MinMaxScaler()

df_raw_x[numeric_columns] = scaler.fit_transform(df_raw_x[numeric_columns])

df_raw_x = pd.get_dummies(df_raw_x, columns=df_raw_x.select_dtypes(include=['object']).columns)
df_raw_x = df_raw_x.astype({column: int for column in df_raw_x.select_dtypes(include=['bool']).columns})

df_train_x, df_test_x, df_train_y, df_test_y = train_test_split(
    df_raw_x, df_raw_y, test_size = 0.3, random_state = 1234, stratify=df_raw_y) 

In [16]:
# 그래디언트 부스팅 모델 생성: GradientBoostingClassifier
gb_uncust = GradientBoostingClassifier(random_state=1234)
gb_uncust.fit(df_train_x, df_train_y)
y_pred = gb_uncust.predict(df_test_x)

print("Test Confusion matrix: \n{}".format(confusion_matrix(df_test_y, y_pred)))
print(classification_report(df_test_y, y_pred, digits=3))

Test Confusion matrix: 
[[470   2]
 [ 25  11]]
              precision    recall  f1-score   support

         0.0      0.949     0.996     0.972       472
         1.0      0.846     0.306     0.449        36

    accuracy                          0.947       508
   macro avg      0.898     0.651     0.711       508
weighted avg      0.942     0.947     0.935       508



In [17]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import randint as sp_randint

estimator = GradientBoostingClassifier( random_state=1234)
# 구하고자 하는 parameter와 범위
param_dist = {
    "learning_rate": [0.1, 0.3, 0.5, 0.7, 1],
    "max_depth": sp_randint(3, 13),
    "min_samples_leaf": sp_randint(5, 30),
    'n_estimators': sp_randint(100, 300)
}

# Randomized Search 객체 생성
random_search_gb = RandomizedSearchCV(estimator, param_distributions=param_dist, n_iter=10, scoring="accuracy", n_jobs=-1, random_state=1234)

# Randomized Search 수행
random_search_gb.fit(df_train_x, df_train_y)

# 결과 출력
print("best estimator model: \n{}".format(random_search_gb.best_estimator_))
print("\nbest parameter: \n{}".format(random_search_gb.best_params_))
print("\nbest score: \n{}".format(random_search_gb.best_score_.round(3)))

best estimator model: 
GradientBoostingClassifier(learning_rate=0.7, max_depth=9, min_samples_leaf=26,
                           n_estimators=216, random_state=1234)

best parameter: 
{'learning_rate': 0.7, 'max_depth': 9, 'min_samples_leaf': 26, 'n_estimators': 216}

best score: 
0.965


In [18]:
gb_uncust=GradientBoostingClassifier(learning_rate=0.7, max_depth=9, min_samples_leaf=26,
                           n_estimators=216, random_state=1234)
gb_uncust.fit(df_train_x, df_train_y)
y_pred = gb_uncust.predict(df_test_x)

print("Train Accuracy: {:.3f}".format(gb_uncust.score(df_train_x, df_train_y)))
print("Test Accuracy: {:.3f}\n".format(gb_uncust.score(df_test_x, df_test_y)))
print("Test Confusion matrix: \n{}".format(confusion_matrix(df_test_y, y_pred)))
print(classification_report(df_test_y, y_pred, digits=3))

Train Accuracy: 1.000
Test Accuracy: 0.969

Test Confusion matrix: 
[[466   6]
 [ 10  26]]
              precision    recall  f1-score   support

         0.0      0.979     0.987     0.983       472
         1.0      0.812     0.722     0.765        36

    accuracy                          0.969       508
   macro avg      0.896     0.855     0.874       508
weighted avg      0.967     0.969     0.968       508



In [ ]:
columns_to_blank = ['Pressure', 'Temp_OXid', 'temp_softbake', 'photoresist_bake', 
                    'Temp_Etching', 'Temp_implantation', 'spin1', 'spin2', 'Source_Power',
                    'temp_HMDS_bake', 'ppm', 'RTA_Temp']

# 대체할 값의 범위 설정
value_ranges = {
    'Pressure': (0.18, 0.192),
    'Temp_OXid': (1294.58, 1348.47),
    'temp_softbake': (95.28, 96.65),
    'photoresist_bake': (5.14, 5.24),
    'Temp_Etching': (68.15, 68.92),
    'Temp_implantation': (97.74, 100.25),
    'spin1': (492.2, 494),
    'spin2': (5172.1, 5194.13),
    'Source_Power': (49.34, 49.98),
    'temp_HMDS_bake': (205.3, 209.5),
    'ppm': (45.89, 49.91),
    'RTA_Temp': (148, 149)
}

# 테스트 데이터프레임 복사
df_test_x_modified = df_raw_x.copy()

# 선택된 열들의 값을 NaN으로 설정
for column in columns_to_blank:
    df_test_x_modified[column] = np.nan

# 나머지 열들의 값들을 설정된 범위 내에서 무작위로 생성하여 대체
for column in df_test_x_modified.columns:
    if column in columns_to_blank:
        min_val, max_val = value_ranges[column]
        random_values = np.random.rand(len(df_test_x_modified))  # [0, 1) 사이의 균일한 확률 분포에서 값 생성
        scaled_values = random_values * (max_val - min_val) + min_val  # 원하는 범위에 맞게 조정
        df_test_x_modified[column] = scaled_values

# 결과 확인
df_test_x_modified

In [ ]:
gb_uncust = GradientBoostingClassifier(learning_rate=0.3, max_depth=9, min_samples_leaf=30,
                                       n_estimators=200, random_state=1234)
gb_uncust.fit(df_train_x, df_train_y)

# 테스트 데이터에 대한 예측 확률
y_proba = gb_uncust.predict_proba(df_test_x)
y_pred = (y_proba[:, 1] > 0.2).astype(int)
# 확률 출력
print(classification_report(df_test_y, y_pred))

In [ ]:
# 테스트 데이터에 대한 예측 확률
y_proba = gb_uncust.predict_proba(df_test_x_modified)
y_pred = (y_proba[:, 1] > 0.2).astype(int)
# 확률 출력

np.sum(y_pred)

In [ ]:
v_feature_name = df_train_x.columns

df_importance = pd.DataFrame()
df_importance ["Feature"] = v_feature_name
df_importance ["Importance"] = gb_uncust.feature_importances_
df_importance .sort_values("Importance", ascending=False, inplace = True)
df_importance .round(3)

In [ ]:
df_importance = pd.DataFrame()
df_importance["Feature"] = v_feature_name
df_importance["Importance"] = gb_uncust.feature_importances_

# df_feature_importance의 테이블을 중요도별로 정렬
df_importance.sort_values("Importance", ascending=False, inplace = True)
df_importance.sort_values("Importance", ascending=True, inplace = True)
coordinates = range(len(df_importance))
top_10_importance = df_importance.nlargest(10, "Importance")

# 시각화
coordinates = range(len(top_10_importance))
plt.barh(y=coordinates[::-1], width=top_10_importance["Importance"])
plt.yticks(coordinates[::-1], top_10_importance["Feature"], fontsize=10)
plt.xlabel("변수 중요도")
plt.ylabel("변수")
plt.title("상위 10개 변수 중요도")
plt.show()

### SMOTE

In [20]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [21]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(df_train_x, df_train_y)
gb_clf = GradientBoostingClassifier(random_state=42)
gb_clf.fit(X_resampled, y_resampled)

y_pred = gb_clf.predict(df_test_x)
print(classification_report(df_test_y, y_pred))

              precision    recall  f1-score   support

         0.0       0.97      0.95      0.96       472
         1.0       0.47      0.61      0.53        36

    accuracy                           0.92       508
   macro avg       0.72      0.78      0.74       508
weighted avg       0.93      0.92      0.93       508



In [22]:
estimator = GradientBoostingClassifier( random_state=1234)
# 구하고자 하는 parameter와 범위
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import randint as sp_randint

estimator = GradientBoostingClassifier( random_state=1234)
# 구하고자 하는 parameter와 범위
param_dist = {
    "learning_rate": [0.1, 0.3, 0.5, 0.7, 1],
    "max_depth": sp_randint(3, 13),
    "min_samples_leaf": sp_randint(5, 30),
    'n_estimators': sp_randint(100, 300)
}

# Randomized Search 객체 생성
random_search_gb = RandomizedSearchCV(estimator, param_distributions=param_dist, n_iter=10,
                                      scoring="accuracy", n_jobs=-1, random_state=1234)

# Randomized Search 수행
random_search_gb.fit(X_resampled, y_resampled)

# 결과 출력
print("best estimator model: \n{}".format(random_search_gb.best_estimator_))
print("\nbest parameter: \n{}".format(random_search_gb.best_params_))
print("\nbest score: \n{}".format(random_search_gb.best_score_.round(3)))

best estimator model: 
GradientBoostingClassifier(learning_rate=0.5, max_depth=9, min_samples_leaf=8,
                           n_estimators=239, random_state=1234)

best parameter: 
{'learning_rate': 0.5, 'max_depth': 9, 'min_samples_leaf': 8, 'n_estimators': 239}

best score: 
0.989


In [24]:
best_grad= GradientBoostingClassifier(learning_rate=0.5, max_depth=9, min_samples_leaf=8,
                           n_estimators=239, random_state=1234)
best_grad.fit(X_resampled, y_resampled)

y_pred = best_grad.predict(df_test_x)
# y_proba = best_grad.predict_proba(df_test_x)
# y_pred = (y_proba[:, 1] > 0.75).astype(int)

print("Confusion matrix: \n{}".format(confusion_matrix(df_test_y, y_pred)))
print(classification_report(df_test_y, y_pred))

Confusion matrix: 
[[468   4]
 [ 11  25]]
              precision    recall  f1-score   support

         0.0       0.98      0.99      0.98       472
         1.0       0.86      0.69      0.77        36

    accuracy                           0.97       508
   macro avg       0.92      0.84      0.88       508
weighted avg       0.97      0.97      0.97       508

